## Version Description
In this version, we deploy a Hybrid PINN to predict two correction factors, i.e the correction factor for both the MTR ( `MTR-600` ) and its corresponding duct part (`TZ-6_zmix`) is predicted by the neural network.

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# import required libraries
import random
import pandas as pd
import numpy as np

from functools import reduce
import sys

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_file,output_notebook
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

In [3]:
# import LTR and corresponding Restrictor dataframe
Restrictor_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_RestrictorData_LTR.csv')
LTR_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_LTRData.csv')
obs_HOVs = pd.read_csv('../data/LTR_dataset.csv', usecols = ['HoV'])[['HoV']]
LTR_data[['AMBP','MIXP']] *= 100 # Convert hPa to Pa
zero_MIXT_hoV_idx = LTR_data.index[LTR_data['MIXT'] == 20].tolist() # Points where originally, MIXT == 0 and substituted with 20° C
LTR_data[['MIXT','AMBT']] += 273.15 # Convert Celcius to Kelvin
print('Total LTR points:',len(LTR_data))

## Calculate Density = (0.5* Pmix + Pamb) / (R * Tmix) where R = 287 is the Gas constant in Ideal Gas Law
idx = LTR_data.columns.get_loc("AMBT")
LTR_data.insert(loc=(idx+1), column='Density(kg/m³)', value=np.round((0.5 * LTR_data['MIXP'] + LTR_data['AMBP'] ) / (LTR_data['MIXT']*287),4))
target_df = reduce(lambda left,right: pd.merge(left,right,left_on='HoV',right_on='HoV'), [LTR_data,Restrictor_data])
target_df['HoV'] = obs_HOVs.values
target_df.head(3)

Total LTR points: 34


,HoV,CAOLH_C66-C68,CAOLH_C68-C70,CAOLH_C70-C72,CAOLH_C72-C74,CAOLH_C74-C76,CAOLH_C76-C78,CAORH_C66-C68,CAORH_C68-C70,CAORH_C70-C72,...,R632_HS1,R633_HS1,R634_HS1,R635_HS1,R636_HS1,R637_HS1,R638_HS1,R639_HS1,R640_HS1,R641_HS1
0,A1,36.071000,38.802000,36.796000,36.794000,39.100000,38.318000,37.055000,39.596000,38.333000,...,41,41,41,41,41,41,41,41,41,41
1,A2,36.061000,38.697000,37.156000,36.738000,38.957000,38.326000,36.501000,40.797000,39.284000,...,41,41,41,41,41,41,41,41,41,41
2,A3,37.504234,40.911735,38.412733,37.859887,36.585486,38.504564,36.161971,37.068225,37.893405,...,41,41,41,41,41,41,41,41,41,41


In [4]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 0 to 33
Data columns (total 58 columns):
HoV               34 non-null object
CAOLH_C66-C68     34 non-null float64
CAOLH_C68-C70     34 non-null float64
CAOLH_C70-C72     34 non-null float64
CAOLH_C72-C74     34 non-null float64
CAOLH_C74-C76     34 non-null float64
CAOLH_C76-C78     34 non-null float64
CAORH_C66-C68     34 non-null float64
CAORH_C68-C70     34 non-null float64
CAORH_C70-C72     34 non-null float64
CAORH_C72-C74     34 non-null float64
CAORH_C74-C76     34 non-null float64
CAORH_C76-C78     34 non-null float64
LAOLH_C66-C68     34 non-null float64
LAOLH_C68-C70     34 non-null float64
LAOLH_C70-C72     34 non-null float64
LAOLH_C72-C74     34 non-null float64
LAOLH_C74-C76     34 non-null float64
LAOLH_C76-C78     34 non-null float64
LAORH_C66-C68     34 non-null float64
LAORH_C68-C70     34 non-null float64
LAORH_C70-C72     34 non-null float64
LAORH_C72-C74     34 non-null float64
LAORH_C74-C76     34 non

In [5]:
MIXP = [1800, 2600, 3400]
# Define Restrictor Labels
MTR = 'R600_HD'
CLRS = ['R610_HS1','R611_HS1','R612_HS1','R613_HS1']
CAORS = ['R620_HS1','R620_HS2','R620_HS3','R621_HS1','R621_HS2','R621_HS3']
MHRS = CLRS + CAORS
LAORS = ['R630_HS1','R631_HS1','R632_HS1','R633_HS1','R634_HS1','R635_HS1','R636_HS1','R637_HS1','R638_HS1','R639_HS1','R640_HS1','R641_HS1']

In [6]:
# Duct Areas
MTR_DuctArea = 0.03464 # in sq.m
MHR_Duct_Areas = {'R610_HS1':0.01767,'R611_HS1':0.01767,'R612_HS1':0.02461,'R613_HS1':0.02461,
                 'R620_HS1':0.01090,'R620_HS2':0.00470,'R620_HS3':0.01085,'R621_HS1':0.01085,'R621_HS2':0.00470,'R621_HS3':0.01090}

## Define Initial Correction factor values from CFD
# Correction Factor for Duct Part `TZ6_zmix:
TZ6_zmix_cf = 1 # Start with initial c_f =1 

#3D Correction Factor for MTR
MTR_cf = 1.1664876228437


#3D Correction Factors for MultiHole Restrictors
MHR_cf = {'R610_HS1':0.740773983383846,'R611_HS1':0.740205245723713,'R612_HS1':0.83879652680329,'R613_HS1':0.81775586281569,
          'R620_HS1':0.757121843184036,'R620_HS2':0.773919779037984,'R620_HS3':0.789028593182055,
          'R621_HS1':0.714811824527744,'R621_HS2':0.866271631648556,'R621_HS3':0.822447592395462}

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [7]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [8]:
target_df[MTR+'_Area'],target_df[MTR+'_Zeta3D'] = zip(*[SHR_Zeta_3D(1,dia,MTR_DuctArea,MTR_cf) for dia in target_df[MTR]])
target_df[[MTR,'R600_HD_Area','R600_HD_Zeta3D']].head()

,R600_HD,R600_HD_Area,R600_HD_Zeta3D
0,148,0.017203,4.949375
1,149,0.017437,4.730821
2,152,0.018146,4.127180
3,154,0.018627,3.764504
4,148,0.017203,4.949375


**For Multi Hole Restrictors:**<br>

    Area_Free = Number_of_Holes * A_Circular(Hole_Diameter)
    f0_f1 = Area_Free / Area_Overall
    l_cross = Thickness / Hole_Diameter
    phi = 0.25 + (0.535 * l_cross ^ 8) / (0.05 + l_cross ^ 7)
    tau = (2.4 - l_cross) * 10 ^ (-phi)
    Zeta_Multi_Hole = (0.5 * (1 - f0_f1) ^ 0.75 + tau * (1 - f0_f1) ^ 1.375 + (1 - f0_f1) ^ 2 + 0.02 * l_cross) / f0_f1 ^ 2

In [9]:
def MHR_Zeta_3D(nr_holes,hole_dia,MHR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Multi Hole Restrictors
    '''
    MHR_New_Area = nr_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MHR_New_Area/MHR_DuctArea
    l_cross = (0.00144*1000)/hole_dia
    phi = 0.25 + (0.535 * l_cross**8) / (0.05 + l_cross**7)
    tau = (2.4 - l_cross) * 10**(-phi)
    zeta_MHR_1D = (0.5 * (1 - f0_f1)**0.75 + tau * (1 - f0_f1)**1.375 + (1 - f0_f1)**2 + 0.02 * l_cross) / f0_f1**2 # 1D Zeta     
    zeta_MHR_3D = zeta_MHR_1D * cf # Zeta with 3D Correction Factor    
    return MHR_New_Area,zeta_MHR_3D

In [10]:
# Calculate Zeta for Multi-Hole Restrictors
for clr in CLRS:
    if (clr == 'R610_HS1') | (clr == 'R611_HS1'):
        hole_dia = 8
    else:
        hole_dia = 10
    MHR_nr_holes = target_df[clr].values
    target_df[clr+'_Area'], target_df[clr+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,hole_dia,MHR_Duct_Areas[clr],MHR_cf[clr]) for ele in MHR_nr_holes])
    
for caor in CAORS:
    CAOR_nr_holes = target_df[caor].values
    target_df[caor+'_Area'],target_df[caor+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,8,MHR_Duct_Areas[caor],MHR_cf[caor]) for ele in CAOR_nr_holes])

In [11]:
# LTR Dataframe col index for Restrictor Zeta and Area values
target_df.iloc[:,58:].head()

,R600_HD_Area,R600_HD_Zeta3D,R610_HS1_Area,R610_HS1_Zeta3D,R611_HS1_Area,R611_HS1_Zeta3D,R612_HS1_Area,R612_HS1_Zeta3D,R613_HS1_Area,R613_HS1_Zeta3D,...,R620_HS2_Area,R620_HS2_Zeta3D,R620_HS3_Area,R620_HS3_Zeta3D,R621_HS1_Area,R621_HS1_Zeta3D,R621_HS2_Area,R621_HS2_Zeta3D,R621_HS3_Area,R621_HS3_Zeta3D
0,0.017203,4.949375,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.00377,8.801675,0.00191,6.836716,0.00568,2.95119
1,0.017437,4.730821,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.00377,8.801675,0.00191,6.836716,0.00568,2.95119
2,0.018146,4.127180,0.006585,7.506119,0.006836,6.741078,0.008954,9.156844,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.00377,8.801675,0.00191,6.836716,0.00568,2.95119
3,0.018627,3.764504,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.00377,8.801675,0.00191,6.836716,0.00568,2.95119
4,0.017203,4.949375,0.007288,5.595443,0.007691,4.756868,0.010210,6.271347,0.010210,6.114035,...,0.00191,6.107864,0.00568,2.786638,0.00377,8.801675,0.00191,6.836716,0.00568,2.95119


**Store Loss Coefficient Values and Names for all Restrictors**:

In [12]:
from FDDN_Lib import zeta_values

In [13]:
# Use Initial Zeta value from the CFD model for the 'TZ-6_zmix' duct part
idx = target_df.columns.get_loc("R600_HD_Zeta3D")
target_df.insert(loc=(idx-1), column='TZ6_zmix_Zeta3D', value=float(zeta_values[0]))

In [14]:
# Gather col_names and their corresponding column indices
zeta_col_names = [col for col in target_df.columns if 'Zeta3D' in col]
zeta_col_names_idx = [target_df.columns.get_loc(col) for col in zeta_col_names if col in target_df]
area_col_names = [col for col in target_df.columns if 'Area' in col]
area_col_names_idx = [target_df.columns.get_loc(col) for col in area_col_names if col in target_df]
pt_var_idx = [target_df.columns.get_loc(col) for col in ['MIXP','AMBP','AMBT'] if col in target_df]
req_flow_rates_idx = [target_df.columns.get_loc(col) for col in ['TZ6_Flow'] if col in target_df]
# print(zeta_col_names_idx)
print(zeta_col_names)

['TZ6_zmix_Zeta3D', 'R600_HD_Zeta3D', 'R610_HS1_Zeta3D', 'R611_HS1_Zeta3D', 'R612_HS1_Zeta3D', 'R613_HS1_Zeta3D', 'R620_HS1_Zeta3D', 'R620_HS2_Zeta3D', 'R620_HS3_Zeta3D', 'R621_HS1_Zeta3D', 'R621_HS2_Zeta3D', 'R621_HS3_Zeta3D']


In [15]:
LTR_zeta_df_col_idx = sorted([0] + req_flow_rates_idx + pt_var_idx + zeta_col_names_idx)
target_df.iloc[:,LTR_zeta_df_col_idx].to_csv('data_output/LTR_Zeta_df.csv', index=False)

In [16]:
LTR_zeta_df = target_df[['HoV','MIXP','AMBP','AMBT','TZ6_Flow']+zeta_col_names]
LTR_zeta_df[['HoV','AMBP','AMBT']+zeta_col_names].tail()

,HoV,AMBP,AMBT,TZ6_zmix_Zeta3D,R600_HD_Zeta3D,R610_HS1_Zeta3D,R611_HS1_Zeta3D,R612_HS1_Zeta3D,R613_HS1_Zeta3D,R620_HS1_Zeta3D,R620_HS2_Zeta3D,R620_HS3_Zeta3D,R621_HS1_Zeta3D,R621_HS2_Zeta3D,R621_HS3_Zeta3D
29,S3,101105.3000,297.285667,0.712987,4.521189,7.506119,6.741078,7.920333,7.721657,9.440922,6.107864,2.786638,8.801675,6.836716,2.95119
30,S4,102356.1438,294.425869,0.712987,4.521189,7.506119,6.741078,7.920333,6.862828,9.440922,6.107864,2.786638,8.801675,6.836716,2.95119
31,S5,102286.7009,294.434538,0.712987,2.839767,7.506119,5.591147,7.920333,6.862828,9.440922,6.107864,2.786638,8.801675,6.836716,2.95119
32,T1,100112.6000,297.133500,0.712987,4.949375,7.506119,6.741078,7.920333,7.721657,9.440922,6.107864,2.786638,8.801675,6.836716,2.95119
33,T2,101099.0000,296.278833,0.712987,4.949375,7.506119,6.741078,7.920333,7.721657,9.440922,6.107864,2.786638,8.801675,6.836716,2.95119


In [17]:
final_df = target_df[['HoV','MIXP','Density(kg/m³)','R600_HD','TZ6_zmix_Zeta3D','R600_HD_Zeta3D','TZ6_Flow']]
final_df.reset_index(drop=True,inplace = True)
# final_df[['MIXP','Density(kg/m³)','R600_HD','TZ6_zmix_Zeta3D','R600_HD_Zeta3D','TZ6_Flow']].head()

### Training data Preparation

In [18]:
from Hybrid_PINN_v1_2 import NormbyMax, DeNormbyMax, PINN

In [19]:
input_features = ['MIXP','Density(kg/m³)','TZ6_zmix_Zeta3D','R600_HD_Zeta3D','TZ6_Flow']
features_max, df_rescaled = NormbyMax(final_df,input_features)
print('Max value of all features:',features_max)

V_max_org = features_max['TZ6_Flow']
print('Maximum TZ6 Flow:',V_max_org)

Max value of all features: {'MIXP': 2632.9689, 'Density(kg/m³)': 1.238, 'TZ6_zmix_Zeta3D': 0.712986788872249, 'R600_HD_Zeta3D': 5.4148802645163, 'TZ6_Flow': 764.0123719999998}
Maximum TZ6 Flow: 764.0123719999998


In [20]:
# Display original and Scaled DataFrame Side-by-side
# print( '{:<60s} {:<20s}'.format('Original Input Dataframe:','Rescaled Input Dataframe:') )
# display_side_by_side(final_df[['HoV']+input_features],df_rescaled)

In [21]:
df_rescaled_inv = DeNormbyMax(df_rescaled,features_max,input_features)

In [22]:
train_data = df_rescaled
# train_data = df_rescaled.iloc[[7,15]]
print("Number of training samples:", len(train_data),'\nSCALED FEATURES:')
train_data

Number of training samples: 34 
SCALED FEATURES:


,MIXP,Density(kg/m³),TZ6_zmix_Zeta3D,R600_HD_Zeta3D,TZ6_Flow
0,0.987478,0.986026,1.0,0.914032,0.966062
1,0.987478,0.987722,1.0,0.873670,0.965658
2,0.989830,0.976575,1.0,0.762192,0.969853
3,0.987440,0.999031,1.0,0.695215,0.958246
4,0.987478,1.000000,1.0,0.914032,0.970963
5,0.987478,0.985299,1.0,0.914032,0.973195
6,0.987478,0.978837,1.0,0.914032,0.968664
7,0.988183,0.967367,1.0,0.728011,0.971982
8,0.985737,0.960420,1.0,0.762192,0.968232
9,0.987478,0.996446,1.0,0.834956,0.968331


In [23]:
# Store indices and values of train data
train_data_idx = train_data.index.values.tolist()
train_R600HD_series = final_df['R600_HD'].iloc[train_data_idx]
X_train = train_data.values

### Predict $c_f$ with Physics Informed Neural Network

In [24]:
# A random seed is used to reproduce the weight initialization values and training performance everytime this notebook is run.
np.random.seed(13)
# Define custom MSE function
MSE = lambda y_hat,y_true:  np.mean((np.asarray(y_hat)-np.asarray(y_true))**2)

**Train the Neural Network**:

In [25]:
### Set the hyperparameters here ###
iterations = 100
learning_rate = 0.01 ### Important Parameter
hidden_nodes = 3 ### Important Parameter
output_nodes = 2
batch_sample_size = 3
MTR_epsi = MTR_cf * 0.01
TZ6_zmix_epsi = TZ6_zmix_cf * 0.01

In [26]:
%%time
N_i = X_train.shape[1]
network = PINN(N_i, hidden_nodes, output_nodes, learning_rate, MTR_epsi, TZ6_zmix_epsi)
MSE_flowloss_hist = []
c_f_hist = {'HoV':[],'MTR_cf_hat':[],'TZ6zmix_cf_hat':[],'FlowRate_Diff_(LTR-FDDN)':[],'ConvergedEpoch':[]}

for ii in range(iterations):
    ## minibatch Gradient Descent
    print("\nEPOCH {} - Length of Training Dataset:".format(ii),len(train_data_idx)) #Check available training data size at the start of every epoch
    batch_idx = np.random.choice(train_data_idx, replace = False, size = batch_sample_size) # train_data_idx
    X_train = train_data[input_features].loc[batch_idx].values
    dia_train = train_R600HD_series.loc[batch_idx]        
    print('Current Indices in Batch:',batch_idx,'\nOld_Zeta:',list(target_df[[MTR+'_Zeta3D']].loc[batch_idx].values[0]))    
    error,V_hat_FDDN_arr,V_true_LTR_arr,break_flag_arr,MTR_cf_arr,TZ6_zmix_cf_arr = network.train(X_train, dia_train, batch_idx, MTR_DuctArea,target_df,zeta_col_names,V_max_org)    
    # Printing out the training progress
    MSE_flowloss = MSE(V_hat_FDDN_arr,V_true_LTR_arr) # MSE between FDDN predicted flow rate and true flow rate on UN-scaled original values
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations))+ "% ... MSE Flowloss: " + str(round(MSE_flowloss,4))[:5] )
    print('... Training Iteration:',ii)       
    sys.stdout.flush()
    MSE_flowloss_hist.append(MSE_flowloss)
#     early_stopping_bf = network.early_stopping(MSE_flowloss_hist, 10) # Call early stopping function to monitor last 10 epochs of the loss array
    if (1 in break_flag_arr): # To remove converged HoVs 
        if(0 in list(batch_idx)):
            row_ids = [0] # Manually include [0] index if it is in the current batch
        else:
            row_ids = list(filter(lambda e: e != 0, list(batch_idx * np.asarray(break_flag_arr)))) #Multiply row_ID list with bf_list and remove 0s
        HoVs = target_df[['HoV']].loc[row_ids].values.tolist()
        MTR_cfs =      list(filter(lambda e: e != 0, list(MTR_cf_arr * np.asarray(break_flag_arr)))) #Multiply MTR_cf_array with bf_list and remove 0s
        TZ6_zmix_cfs = list(filter(lambda e: e != 0, list(TZ6_zmix_cf_arr * np.asarray(break_flag_arr)))) #Multiply TZ6_zmix_array with bf_list and remove 0s
        V_LTR_final = list(filter(lambda e: e != 0, list(V_true_LTR_arr * np.asarray(break_flag_arr))))
        V_FDDN_final = list(filter(lambda e: e != 0, list(V_hat_FDDN_arr * np.asarray(break_flag_arr))))
        print('REMOVING CONVERGED HoVs:',HoVs,'Rows Dropped:',row_ids)
        for hov,cf1,cf2,V_LTR,V_FDDN in zip(HoVs,MTR_cfs,TZ6_zmix_cfs,V_LTR_final,V_FDDN_final):
            c_f_hist['HoV'].append(hov[0])
            c_f_hist['MTR_cf_hat'].append(cf1)
            c_f_hist['TZ6zmix_cf_hat'].append(cf2)
            c_f_hist['FlowRate_Diff_(LTR-FDDN)'].append(V_LTR - V_FDDN )
            c_f_hist['ConvergedEpoch'].append(ii)
        train_data_idx = list(set(train_data_idx).difference(set(row_ids)))
        if (len(train_data_idx) < batch_sample_size): # reduce batch size when available data is less than batch size
            batch_sample_size = batch_sample_size - len(row_ids)        
    if (len(train_data_idx) == 0) or (ii == iterations - 1):
#     if (len(train_data_idx) == 0) or (ii == iterations - 1) or (early_stopping_bf == 1):
        sys.stdout.write('TRAINING PROCESS COMPLETED -')
        if (len(train_data_idx) == 0):
            print(' All HoVs Converged')
        elif (ii == iterations - 1):
            print(' Set Nr. of Training Epochs Reached')
#         elif (early_stopping_bf == 1):
#             print("EARLY STOPPING ACTIVATED")
        break


EPOCH 0 - Length of Training Dataset: 34
Current Indices in Batch: [29  6 14] 
Old_Zeta: [4.521188627716888]

NN output (C_f): [1.18 -0.116]
MTR-Cf: 1.1776117970283975 , TZ6-zmix-Cf: -0.11575339586440804
NN output is -ve. Taking |(c_f)| values: [1.18 0.116]
MTR-Cf: 1.1776117970283975
|TZ6_zmix-Cf|: 0.11575339586440804
New_Zeta for MTR-600: 4.564304807290218
New_Zeta for TZ6_zmix: 0.08253064201842256
FDDN Solver Output: [[77.1 36.8 116 76 40.2 124 12.2 26.3 26.2 26.3 25.7 25.4 11.9 25.9 26.1 25.7 25.3 25.3]]
New_Zeta_epsi for MTR-600: 4.609516693567387
New_Zeta_epsi for TZ6_zmix: -0.010378508496686438
FDDN Solver Output: [[77.2 36.8 116 76.1 40.2 124 12.2 26.3 26.3 26.3 25.7 25.4 11.9 25.9 26.1 25.7 25.4 25.4]]
Row ID: 29
HoV: ['S3']
FlowRate Difference (LTR - FDDN): 0.9202919525474726 l/s
% Diff between flowrates 0.12223996469300519 %

NN output (C_f): [1.14 -0.0989]
MTR-Cf: 1.1351365054697726 , TZ6-zmix-Cf: -0.09891462737199269
NN output is -ve. Taking |(c_f)| values: [1.14 0.0989]
M

FDDN Solver Output: [[73.4 35 111 71.2 37.6 116 11.4 24.6 24.5 24.6 24.1 23.7 11.1 24.2 24.4 24 23.7 23.7]]
New_Zeta_epsi for MTR-600: 6.0440762204002665
New_Zeta_epsi for TZ6_zmix: -1.7514065956267232e-27
FDDN Solver Output: [[73.3 35 110 71 37.5 116 11.4 24.5 24.5 24.6 24 23.7 11.1 24.2 24.4 24 23.7 23.7]]
Row ID: 8
HoV: ['C3']
FlowRate Difference (LTR - FDDN): 31.883611447408043 l/s
Progress: 3.0% ... MSE Flowloss: 2440.... Training Iteration: 3

EPOCH 4 - Length of Training Dataset: 32
Current Indices in Batch: [ 2 15 26] 
Old_Zeta: [4.127180457922134]

NN output (C_f): [0.861 -0.35]
MTR-Cf: 0.8607205340088969 , TZ6-zmix-Cf: -0.3499432758798029
NN output is -ve. Taking |(c_f)| values: [0.861 0.35]
MTR-Cf: 0.8607205340088969
|TZ6_zmix-Cf|: 0.3499432758798029
New_Zeta for MTR-600: 3.045337899971708
New_Zeta for TZ6_zmix: -6.128929614711088e-28
FDDN Solver Output: [[84.6 40.4 128 81.3 42.9 132 13.3 28.8 28.7 28.8 28.2 27.8 13 28.3 28.6 28.1 27.8 27.8]]
New_Zeta_epsi for MTR-600: 3.086

FDDN Solver Output: [[74.8 35.7 113 74 39.1 121 11.8 25.4 25.4 25.4 24.9 24.5 11.5 25 25.2 24.8 24.4 24.5]]
Row ID: 19
HoV: ['H1']
FlowRate Difference (LTR - FDDN): 5.472844597754147 l/s
% Diff between flowrates 0.7455281704029804 %
Progress: 7.0% ... MSE Flowloss: 596.1... Training Iteration: 7
REMOVING CONVERGED HoVs: [['H1']] Rows Dropped: [19]

EPOCH 8 - Length of Training Dataset: 31
Current Indices in Batch: [26 24 18] 
Old_Zeta: [3.337957069017653]

NN output (C_f): [1.3 0.0225]
MTR-Cf: 1.2963823227485676 , TZ6-zmix-Cf: 0.02248736086141944
New_Zeta for MTR-600: 5.5005149651742915
New_Zeta for TZ6_zmix: 2.6736352885438986e-50
FDDN Solver Output: [[74.2 35.4 112 73.2 38.6 119 11.7 25.3 25.2 25.3 24.7 24.4 11.4 24.9 25.1 24.7 24.4 24.4]]
New_Zeta_epsi for MTR-600: 5.550008718009016
New_Zeta_epsi for TZ6_zmix: 8.685935443075093e-52
FDDN Solver Output: [[74 35.3 112 73 38.6 119 11.7 25.2 25.2 25.2 24.7 24.4 11.4 24.8 25 24.6 24.3 24.3]]
Row ID: 26
HoV: ['Q2']
FlowRate Difference (LTR

FDDN Solver Output: [[76.2 36.4 115 73.9 39 120 11.8 25.5 25.5 25.5 25 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
Row ID: 8
HoV: ['C3']
FlowRate Difference (LTR - FDDN): 3.377972854910354 l/s
% Diff between flowrates 0.45768751636222466 %

NN output (C_f): [1.42 0.0377]
MTR-Cf: 1.419857484977522 , TZ6-zmix-Cf: 0.03771774434222512
New_Zeta for MTR-600: 6.024416722973417
New_Zeta for TZ6_zmix: -1.042874419959908e-82
FDDN Solver Output: [[72.1 34.4 109 71.4 37.7 116 11.3 24.5 24.4 24.5 24 23.7 11.1 24.1 24.3 23.9 23.6 23.6]]
New_Zeta_epsi for MTR-600: 6.073910475808141
New_Zeta_epsi for TZ6_zmix: -4.97636149526932e-84
FDDN Solver Output: [[71.9 34.3 108 71.2 37.6 116 11.3 24.4 24.4 24.4 23.9 23.6 11 24 24.2 23.8 23.5 23.5]]
Row ID: 4
HoV: ['A5']
FlowRate Difference (LTR - FDDN): 38.4166357521035 l/s
Progress: 11.0% ... MSE Flowloss: 556.6... Training Iteration: 11
REMOVING CONVERGED HoVs: [['C3']] Rows Dropped: [8]

EPOCH 12 - Length of Training Dataset: 27
Current Indices in Batch: [10 13  4] 

FDDN Solver Output: [[74.8 35.7 113 73.8 39 120 11.8 25.5 25.4 25.5 25 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
New_Zeta_epsi for MTR-600: 5.315461787099705
New_Zeta_epsi for TZ6_zmix: -8.4829453666651035e-109
FDDN Solver Output: [[74.7 35.6 113 73.6 38.9 120 11.8 25.4 25.4 25.4 24.9 24.6 11.5 25 25.2 24.8 24.5 24.5]]
Row ID: 0
HoV: ['A1']
FlowRate Difference (LTR - FDDN): 7.789902707896317 l/s

NN output (C_f): [1.29 -0.0694]
MTR-Cf: 1.2912813399253202 , TZ6-zmix-Cf: -0.06939735413882078
NN output is -ve. Taking |(c_f)| values: [1.29 0.0694]
MTR-Cf: 1.2912813399253202
|TZ6_zmix-Cf|: 0.06939735413882078
New_Zeta for MTR-600: 5.004876515552746
New_Zeta for TZ6_zmix: -5.88693963750727e-110
FDDN Solver Output: [[74.9 35.8 113 73.9 39 120 11.8 25.5 25.5 25.5 25 24.7 11.5 25.1 25.3 24.9 24.6 24.6]]
New_Zeta_epsi for MTR-600: 5.050088401829915
New_Zeta_epsi for TZ6_zmix: 4.6740743119302596e-111
FDDN Solver Output: [[74.8 35.7 113 73.7 38.9 120 11.8 25.5 25.4 25.5 24.9 24.6 11.5 25.1 25.3 24.9 24

...

FDDN Solver Output: [[73.1 34.9 110 72 38.1 117 11.5 24.9 24.8 24.9 24.4 24 11.3 24.5 24.7 24.3 24 24]]
New_Zeta_epsi for MTR-600: 5.702592689588907
New_Zeta_epsi for TZ6_zmix: 0.0
FDDN Solver Output: [[72.9 34.8 110 71.9 38 117 11.5 24.8 24.8 24.8 24.3 24 11.2 24.4 24.6 24.2 23.9 24]]
Row ID: 15
HoV: ['E2']
FlowRate Difference (LTR - FDDN): -42.15987181038554 l/s
Progress: 82.0% ... MSE Flowloss: 959.3... Training Iteration: 82

EPOCH 83 - Length of Training Dataset: 3
Current Indices in Batch: [15 12  9] 
Old_Zeta: [5.702592689588907]

NN output (C_f): [1.34 -0.527]
MTR-Cf: 1.3350268637147857 , TZ6-zmix-Cf: -0.5274360134433229
NN output is -ve. Taking |(c_f)| values: [1.34 0.527]
MTR-Cf: 1.3350268637147857
|TZ6_zmix-Cf|: 0.5274360134433229
New_Zeta for MTR-600: 5.664482702297009
New_Zeta for TZ6_zmix: 0.0
FDDN Solver Output: [[73 34.8 110 72 38 117 11.5 24.9 24.8 24.9 24.4 24 11.3 24.5 24.7 24.3 24 24]]
New_Zeta_epsi for MTR-600: 5.713976455131733
New_Zeta_epsi for TZ6_zmix: -0.0
FDD

FDDN Solver Output: [[74.8 35.7 113 73.8 39 120 11.8 25.5 25.4 25.5 25 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
New_Zeta_epsi for MTR-600: 5.279314035587591
New_Zeta_epsi for TZ6_zmix: 0.0
FDDN Solver Output: [[74.7 35.6 113 73.6 38.9 120 11.8 25.4 25.4 25.5 24.9 24.6 11.5 25 25.3 24.8 24.5 24.5]]
Row ID: 12
HoV: ['D1']
FlowRate Difference (LTR - FDDN): 23.82752820896326 l/s
Progress: 86.0% ... MSE Flowloss: 946.5... Training Iteration: 86

EPOCH 87 - Length of Training Dataset: 3
Current Indices in Batch: [ 9 12 15] 
Old_Zeta: [5.460283805200519]

NN output (C_f): [1.4 -0.623]
MTR-Cf: 1.397578199393442 , TZ6-zmix-Cf: -0.6229190763544133
NN output is -ve. Taking |(c_f)| values: [1.4 0.623]
MTR-Cf: 1.397578199393442
|TZ6_zmix-Cf|: 0.6229190763544133
New_Zeta for MTR-600: 5.416872444851764
New_Zeta for TZ6_zmix: 0.0
FDDN Solver Output: [[73.4 35 111 72.4 38.2 118 11.6 25 25 25 24.5 24.2 11.3 24.6 24.8 24.4 24.1 24.1]]
New_Zeta_epsi for MTR-600: 5.462084331128932
New_Zeta_epsi for TZ6_zmix: -

FDDN Solver Output: [[73.5 35 111 72.4 38.3 118 11.6 25 25 25 24.5 24.2 11.3 24.6 24.8 24.4 24.1 24.1]]
New_Zeta_epsi for MTR-600: 5.450261019877269
New_Zeta_epsi for TZ6_zmix: 0.0
FDDN Solver Output: [[73.3 35 110 72.3 38.2 118 11.6 25 24.9 25 24.4 24.1 11.3 24.6 24.8 24.4 24.1 24.1]]
Row ID: 9
HoV: ['C4']
FlowRate Difference (LTR - FDDN): 23.10410125478836 l/s
Progress: 90.0% ... MSE Flowloss: 933.3... Training Iteration: 90

EPOCH 91 - Length of Training Dataset: 3
Current Indices in Batch: [12 15  9] 
Old_Zeta: [5.266099563178718]

NN output (C_f): [1.41 -0.689]
MTR-Cf: 1.4120032394783724 , TZ6-zmix-Cf: -0.6894107144045285
NN output is -ve. Taking |(c_f)| values: [1.41 0.689]
MTR-Cf: 1.4120032394783724
|TZ6_zmix-Cf|: 0.6894107144045285
New_Zeta for MTR-600: 5.2293658110476935
New_Zeta for TZ6_zmix: 0.0
FDDN Solver Output: [[74.9 35.7 113 73.8 39 120 11.8 25.5 25.5 25.5 25 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
New_Zeta_epsi for MTR-600: 5.272566777754842
New_Zeta_epsi for TZ6_zmix: -

FDDN Solver Output: [[72.9 34.8 110 71.9 38 117 11.5 24.8 24.8 24.9 24.3 24 11.2 24.5 24.7 24.3 24 24]]
New_Zeta_epsi for MTR-600: 5.734652619172896
New_Zeta_epsi for TZ6_zmix: 0.0
FDDN Solver Output: [[72.8 34.7 110 71.8 37.9 117 11.5 24.8 24.7 24.8 24.3 23.9 11.2 24.4 24.6 24.2 23.9 23.9]]
Row ID: 15
HoV: ['E2']
FlowRate Difference (LTR - FDDN): -41.0597427446985 l/s
Progress: 94.0% ... MSE Flowloss: 920.8... Training Iteration: 94

EPOCH 95 - Length of Training Dataset: 3
Current Indices in Batch: [15  9 12] 
Old_Zeta: [5.734652619172896]

NN output (C_f): [1.34 -0.672]
MTR-Cf: 1.3394385412664782 , TZ6-zmix-Cf: -0.6720257316549412
NN output is -ve. Taking |(c_f)| values: [1.34 0.672]
MTR-Cf: 1.3394385412664782
|TZ6_zmix-Cf|: 0.6720257316549412
New_Zeta for MTR-600: 5.6832013302579005
New_Zeta for TZ6_zmix: 0.0
FDDN Solver Output: [[72.9 34.8 110 71.9 38 117 11.5 24.8 24.8 24.9 24.3 24 11.2 24.5 24.7 24.3 24 24]]
New_Zeta_epsi for MTR-600: 5.732695083092625
New_Zeta_epsi for TZ6_zmix

FDDN Solver Output: [[74.9 35.7 113 73.9 39 120 11.8 25.5 25.5 25.5 25 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
New_Zeta_epsi for MTR-600: 5.259243700240313
New_Zeta_epsi for TZ6_zmix: 0.0
FDDN Solver Output: [[74.8 35.7 113 73.7 38.9 120 11.8 25.5 25.4 25.5 24.9 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
Row ID: 12
HoV: ['D1']
FlowRate Difference (LTR - FDDN): 23.099771565134347 l/s
Progress: 98.0% ... MSE Flowloss: 909.6... Training Iteration: 98

EPOCH 99 - Length of Training Dataset: 3
Current Indices in Batch: [ 9 12 15] 
Old_Zeta: [5.450250362938907]

NN output (C_f): [1.4 -0.775]
MTR-Cf: 1.3963033805056437 , TZ6-zmix-Cf: -0.7752687269210548
NN output is -ve. Taking |(c_f)| values: [1.4 0.775]
MTR-Cf: 1.3963033805056437
|TZ6_zmix-Cf|: 0.7752687269210548
New_Zeta for MTR-600: 5.411931375143831
New_Zeta for TZ6_zmix: 0.0
FDDN Solver Output: [[73.4 35 111 72.4 38.2 118 11.6 25 25 25 24.5 24.2 11.3 24.6 24.8 24.4 24.1 24.1]]
New_Zeta_epsi for MTR-600: 5.457143261421
New_Zeta_epsi for TZ6_zmix:

In [27]:
print("MSE_flowloss_history:",MSE_flowloss_hist)

MSE_flowloss_history: [21.858007410801022, 242.36642748010036, 889.8023070500412, 2440.9898090114843, 9785.672939698137, 35576.270753009274, 647.8930665163867, 596.1250766694559, 339.1850508970351, 145.65952956645756, 188.78421234082168, 556.6022551271673, 315.3533850394939, 170.40085192220764, 191.42940171520274, 104.13250684335945, 173.23927511245412, 262.37028980885134, 356.38489941309496, 329.93571381070996, 466.8328229496191, 1403.2875670334731, 582.3455160638774, 195.87856370141483, 201.27107363946936, 925.954618421899, 1285.7333564841756, 566.1863855160947, 241.19299049961697, 727.641687010405, 400.18732116320695, 193.86856688720925, 1001.5898299490377, 413.59806031428866, 436.1685861800667, 256.74316764895156, 697.6222315648605, 345.02924658786264, 142.03280170088905, 24.625286643468673, 235.61187422995215, 477.1773837771719, 115.94098727695496, 585.3997200682446, 1967.2793228981752, 1937.964181127616, 741.3818838047652, 776.8761276435386, 777.0749846229597, 1323.2866203102533,

In [28]:
cf_df = pd.DataFrame(data=c_f_hist)
cf_df.to_csv('data_output/NN_2CFs_Output_miniBatch.csv',index=False)
cf_df

,HoV,MTR_cf_hat,TZ6zmix_cf_hat,FlowRate_Diff_(LTR-FDDN),ConvergedEpoch
0,S3,1.177612,0.115753,0.920292,0
1,C1,1.135137,0.098915,-3.318325,0
2,H1,1.246769,0.004820,5.472845,7
3,P1,1.410349,0.020825,-2.760712,8
4,A6,1.118916,0.048203,1.176009,9
5,A3,1.318591,0.043958,-4.439367,10
6,C3,1.466398,0.008227,3.377973,11
7,D2,1.174526,0.054039,2.204291,12
8,A5,1.182325,0.054882,1.994645,12
9,A2,1.269145,0.051759,4.919804,14


**Plot Training Loss**

In [29]:
hover1    = [('Epoch', '@x'),('MSE FlowLoss', '@y') ]
p1 = figure(width=1600, tooltips=hover1)
p1.line(list(range(1, len(MSE_flowloss_hist)+1)), MSE_flowloss_hist, line_width=2, color='blue', alpha=0.8)
p1.yaxis.axis_label = "MSE FlowRate Loss"
p1.xaxis.axis_label = "Epochs"
show(p1)

### Observations: